# Use XGBoost predict Titanic survived

In [13]:
#coding:utf-8
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.grid_search import GridSearchCV

## 1.Load data and clearning data

In [51]:
def harmonize_data(titanic):#填充空数据 和 把string数据转成integer表示
    #Age
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    titanic.Age[titanic.Age < 16] = 0
    titanic.Age[titanic.Age >= 16] = 1
    # Sex
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 1
    titanic['Sex'] = train['Sex'].astype(int)

    titanic["Embarked"] = titanic["Embarked"].fillna("S")

    titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 1
    titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 2
    titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 3
    titanic['Embarked'] = train['Embarked'].astype(int)

    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic

In [52]:
#read data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

train_df = harmonize_data(train)
test_df =harmonize_data(test)

predictors = ["Pclass","Sex", "Age", "SibSp", "Parch", "Fare","Embarked"]

x_train = train_df[predictors]
y_train = train_df["Survived"]

print x_train.info()
x_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null int32
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null int32
dtypes: float64(2), int32(2), int64(3)
memory usage: 41.8 KB
None


D:\SDE\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\SDE\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,1.0,1,0,7.2500,1
1,1,1,1.0,1,0,71.2833,2
2,3,1,1.0,0,0,7.9250,1
3,1,1,1.0,1,0,53.1000,1
4,3,0,1.0,0,0,8.0500,1


## model select and turning

In [55]:
bst =XGBClassifier(learning_rate=0.01, silent=True, objective='binary:logistic')
# 设置boosting迭代计算次数
param_test = {
    'max_depth': range(3, 12, 1)
}
clf = GridSearchCV(estimator = bst, param_grid = param_test, scoring='accuracy', cv=10)
clf.fit(x_train, y_train)
clf.grid_scores_, clf.best_params_, clf.best_score_

([mean: 0.81033, std: 0.02393, params: {'max_depth': 3},
  mean: 0.80920, std: 0.03037, params: {'max_depth': 4},
  mean: 0.81706, std: 0.04545, params: {'max_depth': 5},
  mean: 0.83277, std: 0.04508, params: {'max_depth': 6},
  mean: 0.84175, std: 0.04498, params: {'max_depth': 7},
  mean: 0.83389, std: 0.04229, params: {'max_depth': 8},
  mean: 0.83277, std: 0.04647, params: {'max_depth': 9},
  mean: 0.83389, std: 0.04686, params: {'max_depth': 10},
  mean: 0.83389, std: 0.04948, params: {'max_depth': 11}],
 {'max_depth': 7},
 0.84175084175084181)

### Prediction

In [56]:
pre_data = clf.predict(test_df[predictors])
print "prediction is:", pre_data

predict_dataframe = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": pre_data
})
predict_dataframe.to_csv('data/xgboost.csv',index=False,encoding="utf-8")

prediction is: [0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0
 0 0 0 1 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0
 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1
 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1
 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0
 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 1 1
 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0
 0 0 1 0 0 1 0 0 1 0 0]
